In [1]:

import matplotlib.pyplot as plt
import iris.quickplot as qplt
import numpy as np
import scipy.stats
from scipy.stats import linregress


In [2]:
from esmvalcore.dataset import Dataset

In [3]:
model_datasets = {
"ACCESS-ESM1-5": 
    Dataset(
    short_name='tos',
    project='CMIP6',
    mip="Omon",
    exp="historical",
    ensemble="r1i1p1f1",
    timerange="19790101/20190101",
    dataset="ACCESS-ESM1-5",
    grid="gn"
)}

obs_datasets = {
"HadISST": 
    Dataset(
    short_name='tos',
    dataset='HadISST',
    mip="Omon",
    project='OBS',
    type='reanaly',
    tier=2),
"ERSSTv5":
    Dataset(
    short_name='tos',
    dataset='NOAA-ERSSTv5',
    mip="Omon",
    project='OBS6',
    type='reanaly',
    tier=2),
"ERA-Interim": 
    Dataset(
    short_name='tos',
    dataset='ERA-Interim',
    mip="Omon",
    project='OBS6',
    type='reanaly',
    timerange="19790101/20190101",
    tier=3)
}

In [4]:
model_datasets = {name: dataset.load() for name, dataset in model_datasets.items()}
obs_datasets = {name: dataset.load() for name, dataset in obs_datasets.items()}

In [5]:
from esmvalcore.preprocessor import anomalies
from esmvalcore.preprocessor import area_statistics
from esmvalcore.preprocessor import climate_statistics
from esmvalcore.preprocessor import monthly_statistics
from esmvalcore.preprocessor import convert_units
from esmvalcore.preprocessor import extract_region
from esmvalcore.preprocessor import extract_month
from esmvalcore.preprocessor import regrid
from esmvalcore.preprocessor import detrend
from esmvalcore.preprocessor import axis_statistics
from esmvalcore.preprocessor import extract_time
from esmvalcore.preprocessor import mask_landsea
import iris

In [30]:
def prepoc1(cube):
    nino34_latext_region = {"start_longitude": 190., "end_longitude": 240., "start_latitude": -5., "end_latitude": 5.}
    cube = regrid(cube, target_grid="1x1", scheme="linear")
    cube = extract_region(cube, **nino34_latext_region)
    cube = anomalies(cube,period='monthly')
    cube = area_statistics(cube,operator='mean')
    cube = extract_month(cube,12) # get DEC
    return cube

def prepoc1(cube):
    nino34_latext_region = {"start_longitude": 190., "end_longitude": 240., "start_latitude": -5., "end_latitude": 5.}
    cube = regrid(cube, target_grid="1x1", scheme="linear")
    cube = extract_region(cube, **nino34_latext_region)
    cube = anomalies(cube,period='monthly')
    cube = area_statistics(cube,operator='mean')
    return cube

In [31]:
model_datasets_prep1 = {name: prepoc1(dataset) for name, dataset in model_datasets.items()}
model_datasets_prep2 = {name: prepoc2(dataset) for name, dataset in model_datasets.items()}

obs_datasets_prep1 = {name: prepoc1(dataset) for name, dataset in obs_datasets.items()}
obs_datasets_prep2 = {name: prepoc2(dataset) for name, dataset in obs_datasets.items()}


In [54]:
import numpy as np
import matplotlib.pyplot as plt
import iris
from iris.time import PartialDateTime
import sacpy as scp

# Extract data from iris cubes
nino34_dec = model_datasets_prep1["ACCESS-ESM1-5"].data  # December only Nino3.4
nino34 = model_datasets_prep2["ACCESS-ESM1-5"].data  # All Nino3.4 data

# Extract the time coordinates from the cubes
time_coord_nino34 = model_datasets_prep2["ACCESS-ESM1-5"].coord('time')

# Convert the time coordinates to years (handling cftime objects)
time_years = np.array([t.year for t in time_coord_nino34.units.num2date(time_coord_nino34.points)])

# Define lead-lag period
leadlagyr = 2

# Select December data
nino34_dec_ct = nino34_dec[leadlagyr:-leadlagyr-1]

# Get the years of the event (matching the time range from nino34_dec_ct)
event_years = time_years[leadlagyr:-leadlagyr-1]

# Create an array to store the years of interest
years_of_interest_array = np.empty((len(event_years), leadlagyr * 3), dtype=int)

# Fill the array with the years of interest for each event year
for i, year in enumerate(event_years):
    years_of_interest_array[i] = [year - 2, year - 1, year, year + 1, year + 2, year + 3]

# Select data for the years of interest from nino34
n34_selected = []

# Set max_length based on the shape of the nino34 array (assuming the time dimension is the first axis)
max_length = nino34.shape[0]


In [57]:
n34_selected = []

# Iterate over each set of years of interest
for i in range(len(years_of_interest_array)):
    # Create an empty list to store the selected data for each year
    selected_data = []
    
    # Loop through each year of interest
    for year in years_of_interest_array[i]:
        # Get indices where time_years matches the current year
        year_indices = np.where(time_years == year)[0]
        
        # Select the data for the matching years and append it
        if len(year_indices) > 0:
            selected_data.append(n34[year_indices])
    
    # Concatenate selected data for this range of years and append to n34_selected
    if selected_data:
        n34_selected.append(np.concatenate(selected_data))

# Convert the list to a numpy array for further processing
n34_selected_array = np.array(n34_selected)
slope = scp.LinReg( n34_dec_ct.values, n34_selected_array).slope

acf1 = autoCorrelation(n34, 36)

plt.plot(np.arange(1, 73) - 36, slope, label='Slope.')
plt.plot(np.arange(1, 74) - 37, acf1, label='Corr.')

plt.xlabel('Lead & Lag Months')
plt.ylabel('Degree C/Corr. Coef')
#plt.title('Plot Title')
plt.legend(); plt.grid()
#plt.savefig('enso_n34_reg_ssta.png', bbox_inches='tight')
plt.show()


IndexError: index 36 is out of bounds for axis 0 with size 36

# note for Felicity

In [ ]:
# here I was using xarray
ds = readnc('n34_Amon_ACCESS-CM2_historical_r1i1p1f1_gn_195001-201412.nc')

# how can I convert to use CUBE to make this code working?

leadlagyr = 2
n34 = ds['n34'].squeeze()
n34_dec = n34[11::12]
n34_dec_ct = n34_dec[leadlagyr:-leadlagyr-1]

event_years = n34_dec_ct.time.dt.year

# Select the data for the years of interest from n34
n34_cases = n34.sel(time=n34.time.dt.year.isin(event_years))

# Create an empty array to store the years of interest
years_of_interest_array = np.empty((len(event_years), leadlagyr*3), dtype=int)

# Fill the array with the years of interest for each event year
for i, year in enumerate(event_years):
    # Ensure that the selected years are not the last or second last year in the n34 dataset
    years_of_interest_array[i] = [year.values - 2, year.values - 1, year.values, year.values + 1, year.values + 2, year.values + 3]


n34_selected = []

for i in range(len(years_of_interest_array)):
    # Select the data for the current year and append it to n34_selected
    n34_selected.append(n34.sel(time=n34['time.year'].isin(years_of_interest_array[i])))

import sacpy as scp
n34_selected_array = np.array(n34_selected)  # Convert the list to a NumPy array
slope = scp.LinReg( n34_dec_ct.values, n34_selected_array).slope

acf1 = autoCorrelation(n34, 36)



In [ ]:
plt.plot(np.arange(1, 73) - 36, slope, label='Slope.')
plt.plot(np.arange(1, 74) - 37, acf1, label='Corr.')

plt.xlabel('Lead & Lag Months')
plt.ylabel('Degree C/Corr. Coef')
#plt.title('Plot Title')
plt.legend(); plt.grid()
plt.savefig('enso_n34_reg_ssta.png', bbox_inches='tight')
plt.show()
